# Proefening 1

Welkom bij de Geonovum workshop! Dit is de eerste oefening.
Als een reminder: Zorg dat je de volgende dingen hebt voorbereid:
- Je hebt git geinstalleerd en de repository gecloned
- Je hebt het install.sh script gerunned, en de conda environment geonovum_pointcloud geactiveerd in dit notebook
- je hebt Cloudcompare, Qgis, of een andere tool om pointclouds te visualiseren geinstalleerd

Zo ja, dan kunnen we aan de slag!

### Package imports

In [7]:
import laspy as lp
import numpy as np
import os

### Handige functies

Onderstaande functies helpen ons in deze workshop. Bij elke functie zie je een korte beschrijving van wat hij doet.

In [16]:
def get_files_in_directory(directory: str):
    """
    Get all files with a specific extension in a directory.
    """

    files = [
        os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(".las")
    ]
    return files


def lasdata_loader(filepaths: str | list, single_array: bool = True):
    """
    Load one or multiple .las or .laz files.
    Can return the data as a single numpy array (if single_array=True)
    or as a list of numpy arrays (if single_array=False).
    """
    if isinstance(filepaths, str):
        filepaths = [filepaths]

    all_points = []
    for filepath in filepaths:
        las = lp.read(filepath)
        coords = np.vstack((las.points.x, las.points.y, las.points.z))
        point_cloud = coords.transpose()
        all_points.append(point_cloud)

    if single_array:
        all_points = np.concatenate(all_points)

    return all_points

### Stap 1: De las data laden

In [ ]:
data_folder = "data/utrecht_subset"  # Pad naar de folder met de las/laz files
lasfiles = get_files_in_directory(data_folder)
print(lasfiles)

['ahn/utrecht_subset/AHN5_C_125000_456000_5_5.las', 'ahn/utrecht_subset/AHN5_C_125000_456000_8_2.las', 'ahn/utrecht_subset/AHN5_C_125000_456000_9_2.las', 'ahn/utrecht_subset/AHN5_C_125000_456000_2_1.las', 'ahn/utrecht_subset/AHN5_C_125000_456000_6_2.las', 'ahn/utrecht_subset/AHN5_C_125000_456000_4_4.las', 'ahn/utrecht_subset/AHN5_C_125000_456000_3_9.las', 'ahn/utrecht_subset/AHN5_C_125000_456000_5_2.las', 'ahn/utrecht_subset/AHN5_C_125000_456000_0_6.las', 'ahn/utrecht_subset/AHN5_C_125000_456000_1_9.las', 'ahn/utrecht_subset/AHN5_C_125000_456000_2_3.las', 'ahn/utrecht_subset/AHN5_C_125000_456000_7_0.las', 'ahn/utrecht_subset/AHN5_C_125000_456000_1_5.las', 'ahn/utrecht_subset/AHN5_C_125000_456000_3_5.las', 'ahn/utrecht_subset/AHN5_C_125000_456000_4_2.las', 'ahn/utrecht_subset/AHN5_C_125000_456000_3_7.las', 'ahn/utrecht_subset/AHN5_C_125000_456000_5_7.las', 'ahn/utrecht_subset/AHN5_C_125000_456000_8_6.las', 'ahn/utrecht_subset/AHN5_C_125000_456000_2_6.las', 'ahn/utrecht_subset/AHN5_C_125

In [ ]:
# We laden eerst 1 van de lasfiles in
lasfile = lasdata_loader(lasfiles[0], single_array=True)